# Communication 1
ELECTRIGRID supplies electricity to a small region from 4 generators running on natural gas. Over the years we have built a network of transmission lines, running from our generators to substation nodes around the region, as shown in the following map:

*   [nodes.csv](https://drive.google.com/file/d/13FvECaoKrowvP4_UPrEanFmsfGqQw_zu/view?usp=sharing) gives the location of each node (km) and the current demand (MW) at that node
*   [grid.csv](https://drive.google.com/file/d/1Pu0NU9JoD353yXliqtPZAW4vPXi7BK55/view?usp=sharing) gives all the connections between the nodes that make up our grid

The red nodes on the map show the locations of our generators. Due to various factors, these generators have different capacities and costs for producing electricity, as shown in the following table:

| Generator Node |  12 |  37 |  23 |  20 |
|----------------|----:|----:|----:|----:|
| Capacity (MW)  | 389 | 792 | 790 | 549 |
| Cost ($/MWh)   |  81 |  73 |  77 |  62 |

Please provide us with the optimal cost for meeting the current demand over a whole day from our generators.

**Correct Answer: $3248616**


## Formulas

### Sets:
* $ N \quad$  nodes {0, 1, ..., 49} 
* $ G \quad$  generator nodes {12, 37, 23, 20}
* $\text{Arcs} \quad$ transmission arcs

### Data:
* $\text{Demand}_n \quad$ the amount of electricity required for node $n \in N$
* $\text{Capacity}_{n} \quad$ maximum capacity(MW) for generator node $n \in G$
* $\text{Cost}_{n} \quad$ cost(\$/MWh) for generator node $n \in G$
* $f_a, t_a \quad$ from and to nodes for $a \in \text{Arcs}$

### Variables: 
* $ X_a \quad$ electricity inflow for $a \in Arcs$

### Objective:
$$ \min \sum_{a \in A  \text{ s.t.} \\ f_a=n} X_a \times \text{Cost}_n \times 24, \quad \forall n \in G $$
 
### Constraints:

$$ \sum_{a \in \text{Arcs} \\ \text{ s.t. } t_a=n} X_a - \sum_{a \in \text{Arcs} \\ \text{ s.t. } f_a=n} X_a = \text{Demand}_n, \quad \forall n \in N \text{ and } n \notin G $$ \

$$ \sum_{a \in \text{Arcs} \\ \text{ s.t. } f_a=n} X_{a} \leq \text{Capacity}_n, \quad \forall n \in G $$ \

## Code

In [ ]:
from gurobipy import *
import numpy as np
# import nodes.csv into a dictionary "nodes"
nodesfile = open('nodes.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes = [ {'x': w[1], 'y': w[2], 'demand': w[3]} for w in nodeslist]
# import grid.csv into a dictionary "arcs"
gridfile = open('grid.csv', 'r')
gridlist = [list(map(int,w.strip().split(','))) for w in gridfile if 'Arc' not in w]
arcs = [ {'Node1': w[1], 'Node2': w[2]} for w in gridlist]

# Sets
N = range(len(nodes)) # nodes
G = [12,37,23,20] # generator nodes
A = range(len(arcs)) # arcs

# Data
capacity = [389,792,790,549]
cost = [81,73,77,62]

f = [arcs[a]['Node1'] for a in A]
t = [arcs[a]['Node2'] for a in A]

m = Model("Electrigrid")

# Variables
X = {}
for a in A:
    X[a] = m.addVar()

# Objective
m.setObjective(quicksum(X[a]*24*cost[G.index(n)] for a in A for n in G if f[a]==n), GRB.MINIMIZE)

# Constraints 
for n in N:
    if n not in G:
        m.addConstr(quicksum(X[a] for a in A if t[a]==n) - 
                    quicksum(X[a] for a in A if f[a]==n) == nodes[n]['demand'])
        
for n in G:
     m.addConstr(quicksum(X[a] for a in A if f[a]==n) <= capacity[G.index(n)])

m.optimize()

# Communication 2
Thank you for your initial estimate. However, we did not mention that our transmission lines actually lose electricity along them. This loss can be estimated as 0.1% per km.

Could you revise your proposal to take this into account? Please provide us with the optimal cost for meeting the current demand over a whole day from our generators.

**Correct Answer: $3498981**

## New Formulas

### New Data:
$fX_a, fY_a \quad$ location (X,Y) for $f_a$ \\
$tX_a, tY_a \quad$ location (X,Y) for $t_a$ \\
$\text{Distance}_a \quad$ distance between the nodes of arc $a \in \text{Arcs}$ which is $\sqrt{(tX_a - fX_a)^2 + (tY_a - fY_a)^2 }$ 
 
### Revised Constraint:
$$ \sum_{\substack{a \in \text{Arcs} \\ \text{s.t. } t_a=n}} X_a (1 - 0.001 \times \text{Distance}_a) - \sum_{\substack{a \in \text{Arcs} \\ \text{s.t. } f_a=n}} X_a = \text{Demand}_n, \quad \forall n \in N \setminus G $$





## Code

In [ ]:
from gurobipy import *
import numpy as np
# import nodes.csv into a dictionary "nodes"
nodesfile = open('nodes.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes = [ {'x': w[1], 'y': w[2], 'demand': w[3]} for w in nodeslist]
# import grid.csv into a dictionary "arcs"
gridfile = open('grid.csv', 'r')
gridlist = [list(map(int,w.strip().split(','))) for w in gridfile if 'Arc' not in w]
arcs = [ {'Node1': w[1], 'Node2': w[2]} for w in gridlist]

# Sets
N = range(len(nodes)) # nodes
G = [12,37,23,20] # generator nodes
A = range(len(arcs)) # arcs

# Data
capacity = [389,792,790,549]
cost = [81,73,77,62]

f = [arcs[a]['Node1'] for a in A]
t = [arcs[a]['Node2'] for a in A]

fX = [nodes[f[a]]['x'] for a in A]
fY = [nodes[f[a]]['y'] for a in A]

tX = [nodes[t[a]]['x'] for a in A]
tY = [nodes[t[a]]['y'] for a in A]

distance = [((tX[a] - fX[a])**2 + (tY[a] - fY[a])**2)**(1/2) for a in A]

m = Model("Electrigrid")

# Variables
X = {}
for a in A:
    X[a] = m.addVar()

# Objective
m.setObjective(quicksum(X[a]*24*cost[G.index(n)] for a in A for n in G if f[a]==n), GRB.MINIMIZE)

# Constraints
for n in N:
    if n not in G:
        m.addConstr(quicksum(X[a]*(1 - 0.001 *distance[a]) for a in A if t[a]==n) - 
                    quicksum(X[a] for a in A if f[a]==n) == nodes[n]['demand'])
        
for n in G:
     m.addConstr(quicksum(X[a] for a in A if f[a]==n) <= capacity[G.index(n)])

m.optimize()

In [ ]:
## Printing
Y = {}
for a in A:
    for n in G:
        if f[a] == n :
            if n not in Y.keys():
                Y[n] = X[a].x
            else:
                Y[n] += X[a].x
for key in Y:
    print('Generator:', key, 'Electricity Production :', Y[key])

# Communication 3
We have realised that your proposal will exceed the limits on some of our transmission lines.\
The following 30 lines can effectively handle any load:

`
[12,13,14,15,16,17,26,27,28,29,34,35,38,39,46,47,128,129,136,137,148,149,150,151,152,153,156,157,188,189]
`
\
However, all of the other lines have a limit of 88 MW. Could you revise your proposal to take this into account? Please provide us with the optimal cost for meeting the current demand over a whole day from our generators. \

**Correct Answer: $3499540**



## New Formulas

### New Set:
* $\text{UnlimitedArcs} \quad$ transmission arcs without limits

### New Constraints:
$$ \sum_{\forall a \notin \text{UnlimitedArcs} \\ \text{ s.t. } f_a=n} X_a \leq 88, \quad n \in N $$

## Code


In [ ]:
from gurobipy import *
import numpy as np
# import nodes.csv into a dictionary "nodes"
nodesfile = open('nodes.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes = [ {'x': w[1], 'y': w[2], 'demand': w[3]} for w in nodeslist]
# import grid.csv into a dictionary "grid"
gridfile = open('grid.csv', 'r')
gridlist = [list(map(int,w.strip().split(','))) for w in gridfile if 'Arc' not in w]
arcs = [ {'Node1': w[1], 'Node2': w[2]} for w in gridlist]

# Sets
N = range(len(nodes)) # nodes
G = [12,37,23,20] # generator nodes
A = range(len(arcs)) # arcs
UnlimitedArcs = [12,13,14,15,16,17,26,27,28,29,34,35,38,39,46,47,128,129,136,137,148,149,150,151,152,153,156,157,188,189]

# Data
capacity = [389,792,790,549]
cost = [81,73,77,62]

f = [arcs[a]['Node1'] for a in A]
t = [arcs[a]['Node2'] for a in A]

fX = [nodes[f[a]]['x'] for a in A]
fY = [nodes[f[a]]['y'] for a in A]

tX = [nodes[t[a]]['x'] for a in A]
tY = [nodes[t[a]]['y'] for a in A]

ArcLimit = 88 # limit for arcs not in UnlimitedArcs

distance = [((tX[a] - fX[a])**2 + (tY[a] - fY[a])**2)**(1/2) for a in A]

m = Model("Electrigrid")

# Variables
X = {}
for a in A:
    X[a] = m.addVar()

# Objective
m.setObjective(quicksum(X[a]*24*cost[G.index(n)] for a in A for n in G if f[a]==n), GRB.MINIMIZE)

# Constraints   
for n in N:
    if n not in G:
        m.addConstr(quicksum(X[a]*(1 - 0.001*distance[a]) for a in A if t[a]==n) - 
                    quicksum(X[a] for a in A if f[a]==n) == nodes[n]['demand'])   
for n in G:
     m.addConstr(quicksum(X[a] for a in A if f[a]==n) <= capacity[G.index(n)])
        
for n1 in N:
    for n2 in N:
        m.addConstr(quicksum(X[a] for a in A if a not in UnlimitedArcs and f[a]==n1 and t[a]==n2) <= ArcLimit)

m.optimize()

# Communication 4
Thank you for helping satisfy this current demand. However, in practice demand changes over the day, and we are concerned about how our network will cope with peak demand times. We have broken the day into six time periods: Midnight to 4am, 4am to 8am, 8am to 12pm, 12pm to 4pm, 4pm to 8pm, and 8pm to midnight.

* [nodes2.csv](https://drive.google.com/file/d/1qHCfmeeuz3N1_x5-0TCMYsu9dLQMHHb6/view?usp=sharing) gives an update of our earlier data with demands for each node (MW) at each of these time periods

Could you revise your proposal to incorporate these changing demands? Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our generators.

**Correct Answer: $3086102**






## New Formulas

### New Set:
* $\text{Periods} \quad$ 6 time periods of a day

### New Data:
* $\text{PeriodDemand}_{n,p} \quad$ the amount of electricity required for node $n \in N$ during the time period $p \in \text{Periods}$

### Revised Varibale:
* $ X_{a,p} \quad$ electricity flow for $a \in Arcs$ during the time period * $p \in \text{Periods}$

### Revised Objective: 
$$ \min \sum_p \sum_{a \in A  \text{ s.t.} \\ f_a=n} X_{a,p} \times 4 \times \text{Cost}_n, \quad \forall n \in G, p \in \text{Periods} $$

### Revised Constraints: 
$$ \sum_{a \in \text{Arcs} \\ \text{ s.t. } t_a=n} X_{a,p} (1-0.001 \times \text{Distance}_a) - \sum_{a \in \text{Arcs} \\ \text{ s.t. } f_a=n} X_{a,p} = \text{PeriodDemand}_{n,p}, \quad \forall n \in N \setminus G, p \in \text{Periods} $$ 

## Code

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 23 14:12:51 2021

@author: ariaq
"""

from gurobipy import *
import numpy as np
# import nodes2.csv into a dictionary "nodes"
nodesfile = open('nodes2.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes = [ {'x': w[1], 'y': w[2], 'D0': w[3], 'D1': w[4], 'D2': w[5], 'D3': w[6], 'D4': w[7], 'D5': w[8]} for w in nodeslist]
# import grid.csv into a dictionary "grid"
gridfile = open('grid.csv', 'r')
gridlist = [list(map(int,w.strip().split(','))) for w in gridfile if 'Arc' not in w]
arcs = [ {'Node1': w[1], 'Node2': w[2]} for w in gridlist]
# import nodes2.csv into a dictionary "nodes2"
nodesfile = open('nodes2.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes2 = [ {'x': w[1], 'y': w[2], 'D0': w[3], 'D1': w[4], 'D2': w[5], 'D3': w[6], 'D4': w[7], 'D5': w[8]} for w in nodeslist]

# Sets
N = range(len(nodes)) # nodes
G = [12,37,23,20] # generator nodes
A = range(len(arcs)) # arcs
UnlimitedArcs = [12,13,14,15,16,17,26,27,28,29,34,35,38,39,46,47,128,129,136,137,148,149,150,151,152,153,156,157,188,189]
P = ['D0','D1','D2','D3','D4','D5']

# Data
capacity = [389,792,790,549]
cost = [81,73,77,62]

f = [arcs[a]['Node1'] for a in A]
t = [arcs[a]['Node2'] for a in A]

fX = [nodes[f[a]]['x'] for a in A]
fY = [nodes[f[a]]['y'] for a in A]

tX = [nodes[t[a]]['x'] for a in A]
tY = [nodes[t[a]]['y'] for a in A]

ArcLimit = 88 # limit for arcs not in UnlimitedArcs

distance = [((tX[a] - fX[a])**2 + (tY[a] - fY[a])**2)**(1/2) for a in A]

m = Model("Electrigrid")

# Variables
X = {}
for a in A:
    for p in P:
        X[a,p] = m.addVar()

# Objective
m.setObjective(quicksum(X[a,p]*4*cost[G.index(n)] for a in A for p in P for n in G if f[a]==n), GRB.MINIMIZE)

# Constraints
for n in N:
    if n not in G:
        for p in P:
            m.addConstr(quicksum(X[a,p]*(1 - 0.001 *distance[a]) for a in A if t[a]==n) - 
                        quicksum(X[a,p] for a in A if f[a]==n) == nodes2[n][p])
for n in G:
    for p in P:
        m.addConstr(quicksum(X[a,p] for a in A if f[a]==n) <= capacity[G.index(n)])
        
for n1 in N:
    for n2 in N:
        for p in P:
            m.addConstr(quicksum(X[a,p] for a in A if a not in UnlimitedArcs and f[a]==n1 and t[a]==n2) <= ArcLimit)

m.optimize()

In [ ]:
Y = {}
for a in A:
    for p in P:
        for n in G:
            if f[a] == n :
                if (n,p) not in Y.keys():
                    Y[n,p] = X[a,p].x 
                else:
                    Y[n,p] += X[a,p].x 
for key in Y:
    print('Generator:', key, 'Electricity Production :', round(Y[key],2))

# Communication 5
It is dangerous for us to make large changes to a generator’s output from one time period to another. We have discussed this with our engineers, and they suggest that each generator’s output should not change by more than 185 MW from one time period to another. Could you revise your proposal to take this into account? Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our generators.\
We look forward to reading your final report.

## New Formulas

### New Constraints:
$$ \sum_{a \in \text{Arcs} \\ f_a=n} X_{a,p_1} - \sum_{a \in \text{Arcs} \\ f_a=n} X_{a,p_2)} \leq 185, $$
$$ \sum_{a \in \text{Arcs} \\ f_a=n} X_{a,p_1} - \sum_{a \in \text{Arcs} \\ f_a=n} X_{a,p_2)} \geq -185, $$
$$\forall n \in G, p_1 \in \text{Periods = \{D0,D1,D2,D3,D4,D5\}}, p_2 \in \text{\{D1,D2,D3,D4,D5,D0\}} $$

## Code

In [ ]:
from gurobipy import *
import numpy as np
# import nodes2.csv into a dictionary "nodes"
nodesfile = open('nodes2.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes = [ {'x': w[1], 'y': w[2], 'D0': w[3], 'D1': w[4], 'D2': w[5], 'D3': w[6], 'D4': w[7], 'D5': w[8]} for w in nodeslist]
# import grid.csv into a dictionary "grid"
gridfile = open('grid.csv', 'r')
gridlist = [list(map(int,w.strip().split(','))) for w in gridfile if 'Arc' not in w]
arcs = [ {'Node1': w[1], 'Node2': w[2]} for w in gridlist]
# import nodes2.csv into a dictionary "nodes2"
nodesfile = open('nodes2.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes2 = [ {'x': w[1], 'y': w[2], 'D0': w[3], 'D1': w[4], 'D2': w[5], 'D3': w[6], 'D4': w[7], 'D5': w[8]} for w in nodeslist]

# Sets
N = range(len(nodes)) # nodes
G = [12,37,23,20] # generator nodes
A = range(len(arcs)) # arcs
UnlimitedArcs = [12,13,14,15,16,17,26,27,28,29,34,35,38,39,46,47,128,129,136,137,148,149,150,151,152,153,156,157,188,189]
P = ['D0','D1','D2','D3','D4','D5']

# Data
capacity = [389,792,790,549]
cost = [81,73,77,62]

f = [arcs[a]['Node1'] for a in A]
t = [arcs[a]['Node2'] for a in A]

fX = [nodes[f[a]]['x'] for a in A]
fY = [nodes[f[a]]['y'] for a in A]

tX = [nodes[t[a]]['x'] for a in A]
tY = [nodes[t[a]]['y'] for a in A]

LIMIT = 88 # limit for arcs not in UnlimitedArcs

distance = [((tX[a] - fX[a])**2 + (tY[a] - fY[a])**2)**(1/2) for a in A]

m = Model("Electrigrid")

# Variables
Xin = {}

for a in A:
    for p in P:
        Xin[a,p] = m.addVar()

# Objective
m.setObjective(quicksum(Xin[a,p]*4*cost[G.index(n)] for a in A for p in P for n in G if f[a]==n), GRB.MINIMIZE)

# Constraints
for n in N:
    if n not in G:
        for p in P:
            m.addConstr(quicksum(Xin[a,p]*(1 - 0.001*distance[a]) for a in A if t[a]==n) - 
                        quicksum(Xin[a,p] for a in A if f[a]==n) == nodes2[n][p])
       
for n in G:
    for p in P:
        m.addConstr(quicksum(Xin[a,p] for a in A if f[a]==n) <= capacity[G.index(n)])
        
for n1 in N:
    for n2 in N:
        for p in P:
            for a in A:
                 if a not in UnlimitedArcs and f[a]==n1 and t[a]==n2:
                     m.addConstr(Xin[a,p] <= LIMIT)

for n in G:
    for p in P:
        if p != 'D0':
            m.addConstr(quicksum(Xin[a,P[P.index(p)]] for a in A if f[a]==n) 
                        - quicksum(Xin[a,P[P.index(p) - 1]] for a in A if f[a]==n) <= 185)
            
            m.addConstr(quicksum(Xin[a,P[P.index(p)]] for a in A if f[a]==n) 
                        - quicksum(Xin[a,P[P.index(p)-1]] for a in A if f[a]==n) >= -185)
              
m.optimize()

In [ ]:
Y = {}
for a in A:
    for p in P:
        for n in G:
            if f[a] == n :
                if (n,p) not in Y.keys():
                    Y[n,p] = Xin[a,p].x 
                else:
                    Y[n,p] += Xin[a,p].x 
for key in Y:
    print('Generator:', key, 'Electricity Production :', round(Y[key],2))

# Printings for Final Reports

## Comm1-3

In [ ]:
## Printing
# Electricity Production
Y = {}
for a in A:
    for n in G:
        if f[a] == n :
            if n not in Y.keys():
                Y[n] = X[a].x
            else:
                Y[n] += X[a].x

# Create result list
table = []
for g in list(Y.keys()):
    D = {}
    D['Generator'] = g
    EP = list(Y.values())[list(Y.keys()).index(g)]
    D['Electricity Production (MW/h)'] = round(EP,2)
    D['Cost ($/h)'] = round(cost[G.index(g)]*EP,2)
    table.append(D)
print(table)

# Write into csv
import csv 
fields = ['Generator', 'Electricity Production (MW/h)', 'Cost ($/h)'] 
filename = "comm_print.csv"
with open(filename, 'w', newline="") as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = fields) 
    writer.writeheader() 
    writer.writerows(table)

We can import the csv file on the website [Table Generator](https://www.tablesgenerator.com/latex_tables) to get our latex table easily. 

## Comm4-5

In [ ]:
Y = {}
for a in A:
    for p in P:
        for n in G:
            if f[a] == n :
                if (n,p) not in Y.keys():
                    Y[n,p] = X[a,p].x 
                else:
                    Y[n,p] += X[a,p].x 

import pandas as pd
table = []
for i in range(len(list(Y.keys()))):
    D = {}
    D['Generator'] = list(Y.keys())[i][0]
    D['Period'] = list(Y.keys())[i][1]
    EP = list(Y.values())[i]
    D['Electricity Production (MW/h)'] = round(EP,2)
    D['Cost ($/h)'] = round(cost[G.index(D['Generator'])]*EP,2)
    table.append(D)

df = pd.DataFrame(table)
df = df.set_index(['Generator', 'Period'])
print(df)
df.to_csv('comm4_print.csv', index='True')

## Sensitivity analysis

In [ ]:
## Sensitive Analysis
print('\nCapacity')
for c in m.getConstrs():
    if c.ConstrName == 'capacity' :
        if c.pi != 0:
            print(G[capacity.index(c.RHS)], c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))
            
print("\nConstraints")
for c in m.getConstrs():
    if c.ConstrName == 'LIMIT' :
        if c.pi != 0:
            print(c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))
            
print("\nConstraints")
for c in m.getConstrs():
    if c.ConstrName == 'PosChange' :
        if c.pi != 0:
            print(c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))
print("\nConstraints")
for c in m.getConstrs():
    if c.ConstrName == 'NegChange' :
        if c.pi != 0:
            print(c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))

# Section A Code

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 23 14:15:09 2021

@author: Danny Wang (46443753), Ruyun Qi (44506065)
"""

from gurobipy import *
import pandas as pd
import numpy as np

## import data

# import nodes2.csv into a dictionary "nodes"
nodesfile = open('nodes2.csv', 'r')
nodeslist = [list(map(int,w.strip().split(','))) for w in nodesfile if 'Node' not in w]
nodes2 = [ {'x': w[1], 'y': w[2], 'D0': w[3], 'D1': w[4], 'D2': w[5], 'D3': w[6], 'D4': w[7], 'D5': w[8]} for w in nodeslist]

# import grid.csv into a dictionary "grid"
gridfile = open('grid.csv', 'r')
gridlist = [list(map(int,w.strip().split(','))) for w in gridfile if 'Arc' not in w]
arcs = [ {'Node1': w[1], 'Node2': w[2]} for w in gridlist]


## Model

# Sets
N = range(len(nodes2)) # nodes
G = [12,37,23,20] # generator nodes
A = range(len(arcs)) # arcs
UnlimitedArcs = [12,13,14,15,16,17,26,27,28,29,34,35,38,39,46,47,128,129,136,137,148,149,150,151,152,153,156,157,188,189]
P = ['D0','D1','D2','D3','D4','D5'] # periods

# Data
capacity = [389,792,790,549]
cost = [81,73,77,62]

f = [arcs[a]['Node1'] for a in A] # from node
t = [arcs[a]['Node2'] for a in A] # to node

fX = [nodes2[f[a]]['x'] for a in A]
fY = [nodes2[f[a]]['y'] for a in A]

tX = [nodes2[t[a]]['x'] for a in A]
tY = [nodes2[t[a]]['y'] for a in A]

distance = [((tX[a] - fX[a])**2 + (tY[a] - fY[a])**2)**(1/2) for a in A]

ArcLimit = 88 # limit for arcs not in UnlimitedArcs

ChangeLimit = 185 # change limit for a generator between 2 periods

m = Model("Electrigrid")

# Variables
X = {}
for a in A:
    for p in P:
        X[a,p] = m.addVar()

# Objective
m.setObjective(quicksum(X[a,p]*4*cost[G.index(n)] for a in A for p in P for n in G if f[a]==n), GRB.MINIMIZE)

# Demand constraint
for n in N:
    if n not in G:
        for p in P:
            m.addConstr(quicksum(X[a,p]*(1 - 0.001*distance[a]) for a in A if t[a]==n) - 
                        quicksum(X[a,p] for a in A if f[a]==n) 
                        == nodes2[n][p], name='demand')

# Capacity constraint
for n in G:
    for p in P:
        m.addConstr(quicksum(X[a,p] for a in A if f[a]==n) 
                    <= capacity[G.index(n)], name='capacity')

# ArcLimit constraint        
for n1 in N:
    for n2 in N:
        for p in P:
            for a in A:
                 if a not in UnlimitedArcs and f[a]==n1 and t[a]==n2:
                     m.addConstr(X[a,p] <= ArcLimit, name='ArcLimit')

# ChangeLimit constraints
for n in G:
    for p in P:
        if p != 'D0':
            m.addConstr(quicksum(X[a,P[P.index(p)]] for a in A if f[a]==n) 
                        - quicksum(X[a,P[P.index(p) - 1]] for a in A if f[a]==n) 
                        <= 185, name='PosChange')
            
            m.addConstr(quicksum(X[a,P[P.index(p)]] for a in A if f[a]==n) 
                        - quicksum(X[a,P[P.index(p)-1]] for a in A if f[a]==n) 
                        >= -185, name='NegChange')
  
m.optimize()


## Printings

# electricity production of each generator during each period
Y = {}
for a in A:
    for p in P:
        for n in G:
            if f[a] == n :
                if (n,p) not in Y.keys():
                    Y[n,p] = X[a,p].x 
                else:
                    Y[n,p] += X[a,p].x 
table = []
for i in range(len(list(Y.keys()))):
    D = {}
    D['Generator'] = list(Y.keys())[i][0]
    D['Period'] = list(Y.keys())[i][1]
    EP = list(Y.values())[i]
    D['Electricity Production (MW/h)'] = round(EP,2)
    D['Cost ($/h)'] = round(cost[G.index(D['Generator'])]*EP,2)
    table.append(D)
df = pd.DataFrame(table)
df = df.set_index(['Generator','Period'])
print(df)


## Sensitive Analysis
G20Save = []
print('\nCapacity')
for c in m.getConstrs():
    if c.ConstrName == 'capacity' :
        if c.pi != 0:
            print(G[capacity.index(c.RHS)], c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))
            if G[capacity.index(c.RHS)] == 20:
                G20Save.append(c.pi)
print('Generator20 Average Save: ', np.mean(G20Save))                
                
ArcLimitSave = []
print("\nConstraints")
for c in m.getConstrs():
    if c.ConstrName == 'ArcLimit' :
        if c.pi != 0:
            ArcLimitSave.append(c.pi)
            print(c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))
print('ArcLimit Average Save: ', np.mean(ArcLimitSave))
            
print("\nConstraints")
for c in m.getConstrs():
    if c.ConstrName == 'PosChange' :
        if c.pi != 0:
            print(c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))

print("\nConstraints")
for c in m.getConstrs():
    if c.ConstrName == 'NegChange' :
        if c.pi != 0:
            print(c.ConstrName, c.slack, c.pi, (c.SARHSLow, c.RHS, c.SARHSUp))